In [1]:
import json
import gzip
import os
import time

import requests
import pandas as pd

import jsonpickle

import config
from document import Document

indexer_url = config.indexer_url
text_processing_url = config.text_processing_url
ranking_url = config.ranking_url
snippets_url = config.snippets_url
res_page_form_url = config.res_page_form_url


In [2]:
def load_data(data_path):
    """
    Load all data to pandas.DataFrame
    
    :param str data_path: Path to folder with data 
    :return pd.DataFrame data:
    """
    docs_info = []
    docs_text = []
    file_names = os.listdir(data_path)

    for file in file_names:
        if "text" in file:
            with gzip.open(os.path.join(data_path, file), "rb") as f:
                for line in f:
                    vacancy = json.loads(line)
                    docs_text.append(vacancy)
        else:
            with gzip.open(os.path.join(data_path, file), "rb") as inf:
                for line in inf:
                    vacancy = json.loads(line)
                    docs_info.append(vacancy)

    assert len(docs_info) == len(docs_text)
    assert "id" in docs_info[0].keys()
    assert "id_job" in docs_text[0].keys()

    docs_info = pd.DataFrame(docs_info)
    docs_text = pd.DataFrame(docs_text)
    docs_info.drop_duplicates(["id"], inplace=True)
    docs_text.drop_duplicates(["id_job"], inplace=True)
    data = docs_info.merge(docs_text, left_on='id', right_on='id_job', how='outer')
    
    return data


def add_documents_to_index(documents):
    """
    Add the document from pd.DataFrame to index.
    
    :param pandas.DataFrame documents:
    """
    for index, row in documents.iterrows():
        doc_id = row["id"]
        title = row["title"]
        text = row["text"]
        language = row["lang_text"]
        doc = Document(doc_id, title, text, language)
        r = requests.post(text_processing_url + config.NORMALIZE_DOC_PATH,
                          json=jsonpickle.encode(doc))
        status_normalizer = r.status_code
        doc_normalized = jsonpickle.decode(r.text)
        
        r = requests.post(indexer_url + config.INDEXER_PATH,
                          json=jsonpickle.encode(doc_normalized))
        status_indexer = r.status_code

        if status_indexer != 200 or status_normalizer != 200:
            print(" Request number", index, ": ")
            print("error")
    print("dataset adding finished!")
    
    
# -------##--------##--------##--------##--------##--------##--------##--------#

In [3]:
dataset = load_data(config.dataset_dir)
print(len(dataset))

90081


In [4]:
dataset.head()

,campaigns,checksum,company_name,company_name_hash64,date_created_x,date_expired,date_updated,emails,emails_src,html_desc_mode,...,title,title_normalized,uid,url,date_created_y,id_job,text,text_hash64,text_word_count,title_hash64
0,NaN,-4347516908546459650,ТОРГОВАЯ КОМПАНИЯ МИНСК КРИСТАЛЛ ТРЕЙД,-1407800922893091652,22.01.2017 5:08:55,,21.01.2017 0:00:00,,,1,...,Мерчендайзер,мерчендайзер,-9212801961204319130,https://osipovichi.jobs.tut.by/vacancy/19463412,22.01.2017 5:08:55,-6976808890660704963,Обязанности: -Выкладка продукции согласно стан...,8576765356311114568,,-1714090597579181297
1,NaN,7224660365041947041,,,22.01.2017 15:59:00,,22.01.2017 0:00:00,,,1,...,работа на дому,работа на дому,-9209947671975399058,http://slanet.by/rabota_na_domu/~8634706/~256/,22.01.2017 15:59:00,8101093402702063722,\r\n Приглашаю в совместный бизнес. \r\n Если ...,-2798404691997887952,,-9181459132912603677
2,NaN,-311535052514607523,ТиАнСи,4079004396763991616,22.01.2017 23:08:36,,22.01.2017 0:00:00,,,1,...,Приемщик-администратор заказов,приемщик администратор заказов,-9206403750529000147,https://minsk.jobs.tut.by/vacancy/19464616,22.01.2017 23:08:36,-4134875733109679071,Обязанности:\r\n~Приемка одежды в швейный ремо...,-3558390396120362415,,-2393853823647787673
3,NaN,7737366870595005744,ИП Волченко Наталья Викторовна,3209921620681361505,22.01.2017 21:46:08,,22.01.2017 0:00:00,,,2,...,Преподаватель в детский центр,преподаватель в детский центр,-9188400109306636683,http://rabota.by/vacancy/view/605376/,22.01.2017 21:46:08,3640860530202637173,Проводить занятия по декоративно-прикладному и...,-4917747260559578006,,7056557067088382888
4,NaN,-8053228559258502755,"ЗАО ""АТЛАНТ""",3024363518202159677,22.01.2017 5:48:41,,21.01.2017 0:00:00,,,1,...,Слесарь по обслуживанию и ремонту газоиспользу...,слесарь по обслуживанию и ремонту газоиспользу...,-9148989982408761561,http://minsk.regiony.by/работа/вакансии/#!jobs...,22.01.2017 5:48:41,6129885995940329470,Образование:Профессионально-техническоеХаракте...,4098862302437213354,,8463144365626593477


### Run cell below to add documents to index and save it

set variable n_docs to a small value for debugging

In [11]:
# Run this cell only once
# Adding may takes from 15min (for 30k docs) to 1 hour
# Index with 30k documents takes about 2-3 Gb RAM
# 5k documents enough for testing during development

n_docs = 500
start = time.time()
add_documents_to_index(dataset.loc[:n_docs])
print("Adding time =", time.time() - start)

# Save current status of index

r = requests.post(indexer_url + "save_index")
print(r.status_code)


dataset adding finished!
Adding time = 59.88505673408508
200


## Test all services

In [122]:
print("Normalize the document.")
row = dataset.loc[1001, :]
doc_id = row["id"]
title = row["title"]
text = row["text"]
language = row["lang_text"]
doc = Document(doc_id, title, text, language)

r = requests.post(text_processing_url + config.NORMALIZE_DOC_PATH, json=jsonpickle.encode(doc))
status_normalizer = r.status_code
doc_normalized = jsonpickle.decode(r.text)
print(doc_normalized)


Normalize the document.
{'id': '5896349877670049350', 'language': 'ru', 'snippet': None, 'text': 'Режим работы:Полный рабочий деньЗарплата:133.00 руб.Вакансия:Инструктор по лечебной физкультуре (Инструктор по ЛФК городской поликлиники на 0,5 ставки временно)Адрес:Гродненская область, г. Слоним, , ул. Войкова, 51 А Телефоны:8-015-62-3-24-13 8-015-62-3-24-13\r\n\r\n', 'text_normalized': 'реж работ полн рабоч инструктор по лечебн физкультур инструктор по лфк городск поликлиник на ставк времен адрес гродненск област слон ул войков а', 'title': 'Инструктор по лечебной физкультуре', 'title_normalized': 'инструктор по лечебн физкультур'}


In [123]:
print("\nAdding the document to index")
r = requests.post(indexer_url + config.INDEXER_PATH,
                  json=jsonpickle.encode(doc_normalized))
print(r.status_code)
print(r.text)



Adding the document to index
200
document already exist in index.


In [124]:
print("\nNormilize query \"водитель\".")
query = "водитель"
r = requests.post(text_processing_url + config.NORMALIZE_QUERY_PATH,
                  json=query)
print(r.status_code)
search_query = r.text
print(search_query)



Normilize query "водитель".
200
водител


In [130]:
print("\nSearch in index.")
r = requests.post(indexer_url + config.SEARCH_PATH, json=search_query)
print(r.status_code)
search_result = jsonpickle.decode(r.text)
print(search_result["documents"][:2])



Search in index.
200
[Title: Водитель погрузчика
Text: Режим работы:Полный рабочий деньЗарплата:400.00 руб.Вакансия:Водитель погрузчика (Погрузка вагонов г.Калинковичи.)Адрес:Гомельская область, г. Мозырь, , ул. Советская, 25Телефоны:8-0236-32-00-45, 32-05-98-юрист 8-0236-32-00-45, 8-0236-32-04-43


Snippet: None
, Title: Водитель автомобиля
Text: Образование:Профессионально-техническоеХарактер работы:ПостояннаяЗарплата:500.00 руб.Вакансия:Водитель автомобиля (вывозка леса)Адрес:Витебская область, г. Браслав, , ул. Дачная, 1А Телефоны:64585 8(02153) 64585 8(02153) 64585


Snippet: None
]


In [131]:
documents = search_result["documents"]
terms = search_result["terms"]


In [135]:
print("\nRanking the document")
docs_query = {"documents": documents, "query": search_query}
r = requests.post(ranking_url + config.RANK_PATH,
                  json=jsonpickle.encode(docs_query))
print(r.status_code)
rank_result = jsonpickle.decode(r.text)
print("len = ", len(rank_result))
print(rank_result[:2])



Ranking the document
200
len =  8
[(0.0, Title: Водитель в службу доставки еды
Text: Водитель с личным л/а для доставки готовой продукции (еда) по г. Барановичи. Режим работы с 17.00 до 22.00. Оплата 3р. за одну доставку.

Snippet: None
), (0.0, Title: Водитель с личным автомобилем бус ищет работу или подработку
Text: Водитель с личным автомобилем бус ищет работу или подработку 

Snippet: None
)]


In [136]:
# FOR TF-IDF Method
print("\nGet snippets for TF-IDF Method.")
docs_query = {"documents": [i[1] for i in rank_result], "terms": terms}
r = requests.post(snippets_url + config.SNIPPETS_PATH,
                  json=jsonpickle.encode(docs_query))
print(r.status_code)
rank_result = jsonpickle.decode(r.text)
print(rank_result[:2])



Get snippets for TF-IDF Method.
200
[Title: Водитель в службу доставки еды
Text: Водитель с личным л/а для доставки готовой продукции (еда) по г. Барановичи. Режим работы с 17.00 до 22.00. Оплата 3р. за одну доставку.

Snippet: Водитель с личным л/а для доставки готовой продукции (еда) по г. Барановичи. Режим работы с 17.00 до 22.00. Оплата 3р. за одну доставку.

, Title: Водитель с личным автомобилем бус ищет работу или подработку
Text: Водитель с личным автомобилем бус ищет работу или подработку 

Snippet: Водитель с личным автомобилем бус ищет работу или подработку 

]


In [111]:
# FOR BM-25 Method
print("\nGet snippets for BM-25 Method.")
docs_query = {"documents": [i[0] for i in rank_result], "terms": terms}
r = requests.post(snippets_url + config.SNIPPETS_PATH,
                  json=jsonpickle.encode(docs_query))
print(r.status_code)
rank_result = jsonpickle.decode(r.text)
print(rank_result[:2])


Get snippets for BM-25 Method.


TypeError: 'Document' object does not support indexing

In [137]:
print("\nSERP results")
docs_query = {"documents": rank_result, "query": search_query}
r = requests.post(res_page_form_url + config.RESULT_PAGE_PATH,
                  json=jsonpickle.encode(docs_query))
print(r.status_code)
search_result = r.text
print(search_result)



SERP results
200
Водитель в службу доставки еды
document id a.k.a. url: -2566770001090623967
Водитель с личным л/а для доставки готовой продукции (еда) по г. Барановичи. Режим работы с 17.00 до 22.00. Оплата 3р. за одну доставку.

....................................................................................................
Водитель с личным автомобилем бус ищет работу или подработку
document id a.k.a. url: -2635371348590851677
Водитель с личным автомобилем бус ищет работу или подработку 

....................................................................................................
Водитель автомобиля
document id a.k.a. url: -3806050872766449737
Режим работы:Полный рабочий деньЗарплата:400.00 руб.Разряд / категория:Второй (II, 2)Вакансия:Водитель автомобиля (Наличие категории "Е")Адрес:Могилевская область, г. Быхов, , м-р Колос, д.7б Телефоны:802231-54738 54738


....................................................................................................
Водитель 